In [1]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
df = pd.read_csv('TRAIN.csv')
test = pd.read_csv('TEST.csv')

In [3]:
#df = df.drop(columns = ['ID', "Order"])

In [4]:
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d', errors = 'coerce')
df.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [5]:
df['Date'] = df['Date'].dt.dayofweek
df.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Order,Sales
0,T1000001,1,S1,L3,R1,0,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,0,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,0,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,0,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,0,1,Yes,62,45614.52


In [9]:
test['Date'] = pd.to_datetime(test['Date'], format = '%Y-%m-%d', errors = 'coerce')
test['Date'] = test['Date'].dt.dayofweek

In [10]:
test

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount
0,T1188341,171,S4,L2,R3,5,0,No
1,T1188342,172,S1,L1,R1,5,0,No
2,T1188343,173,S4,L2,R1,5,0,No
3,T1188344,174,S1,L1,R4,5,0,No
4,T1188345,170,S1,L1,R2,5,0,No
...,...,...,...,...,...,...,...,...
22260,T1210601,186,S2,L5,R2,2,0,No
22261,T1210602,11,S4,L2,R1,2,0,No
22262,T1210603,185,S1,L1,R3,2,0,Yes
22263,T1210604,69,S1,L1,R4,2,0,No


In [6]:
submit = pd.DataFrame()
df.to_csv('Train_day.csv', index = False)
print("file saved")

file saved


In [11]:

test.to_csv('Test_day.csv', index = False)
print("file saved")

file saved


# training auto ml 

In [ ]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(df, test_size=0.05, random_state=223)

In [ ]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": 0.4,
    "enable_early_stopping": True,
    "primary_metric": 'r2_score',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             training_data=df,
                             label_column_name="Sales",
                             **automl_settings)

In [ ]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "Sales_prediction")
local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

In [ ]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

y_test = x_test.pop("Sales")

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])

In [ ]:
test_id = test.iloc['ID']
test = test.drop(columns = ['ID'])

In [ ]:
pred = fitted_model.predict(test)


In [ ]:
submit = pd.DataFrame()
submit['ID'] = test_id['ID']
submit['Sales'] = pred
submit.to_csv('Submission_automl.csv', index = False)
print("file saved")